In [1]:
%pip install evaluate

In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType, PeftModel, PeftConfig
from evaluate import load
import numpy as np 

In [5]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
model_name = "gpt2"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
dataset = load_dataset("imdb")

Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 430939.06 examples/s]


In [9]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id


In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map: 100%|██████████| 50000/50000 [00:05<00:00, 8868.17 examples/s]


In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = np.mean(predictions == labels)
    return {"accuracy": accuracy}

In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=1,
)

/home/hellfire/miniconda3/envs/torchenv/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [14]:
evaluation_results = trainer.evaluate()
print("Initial model performance:")
print(f"Accuracy: {evaluation_results['eval_accuracy']:.4f}")


Initial model performance:
Accuracy: 0.4998


# Performing Parameter-Efficient Fine-Tuning

In [15]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
)

In [16]:
# Create PEFT model
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 296,448 || all params: 124,737,792 || trainable%: 0.2377


/home/hellfire/miniconda3/envs/torchenv/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [17]:
# Set up Trainer for fine-tuning
peft_training_args = TrainingArguments(
    output_dir="./peft_results",
    learning_rate=1e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/home/hellfire/miniconda3/envs/torchenv/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [19]:
peft_trainer.train()

2024/10/15 18:06:17 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id d54a61c61cb6452fb0afcda83a0f0b98: Failed to log run data: Exception: Changing param values is not allowed. Param with key='logging_dir' was already logged with value='./results/runs/Oct15_17-58-48_HellFire' for run ID='d54a61c61cb6452fb0afcda83a0f0b98'. Attempted logging new value './peft_results/runs/Oct15_18-05-53_HellFire'.
2024/10/15 18:06:17 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id d54a61c61cb6452fb0afcda83a0f0b98: Failed to log run data: Exception: Changing param values is not allowed. Param with key='output_dir' was already logged with value='./results' for run ID='d54a61c61cb6452fb0afcda83a0f0b98'. Attempted logging new value './peft_results'.
2024/10/15 18:20:43 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id d54a61c61cb6452fb0afcda83a0f0b98: Failed to log run data: Exception: Changing param values is not allowed. Param with key='logging_dir' was already logged wit

Epoch,Training Loss,Validation Loss,Accuracy
1,0.245200,0.223644,0.931920


TrainOutput(global_step=3125, training_loss=0.3267373571777344, metrics={'train_runtime': 419.8418, 'train_samples_per_second': 59.546, 'train_steps_per_second': 7.443, 'total_flos': 6555185971200000.0, 'train_loss': 0.3267373571777344, 'epoch': 1.0})

In [20]:
peft_model.save_pretrained("./peft_imdb_model")

# Performing Inference with a PEFT Model


In [21]:
peft_config = PeftConfig.from_pretrained("./peft_imdb_model")


In [22]:
base_model = AutoModelForSequenceClassification.from_pretrained(peft_config.base_model_name_or_path).to(device)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
peft_model = PeftModel.from_pretrained(base_model, "./peft_imdb_model")
peft_model.config.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

In [28]:
peft_eval_args = TrainingArguments(
    output_dir="./peft_eval_results",
    evaluation_strategy="epoch",
    num_train_epochs=1,
)

/home/hellfire/miniconda3/envs/torchenv/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
peft_eval_trainer = Trainer(
    model=peft_model,
    args=peft_eval_args,
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [30]:
peft_evaluation_results = peft_eval_trainer.evaluate()
print("PEFT model performance:")
print(f"Accuracy: {peft_evaluation_results['eval_accuracy']:.4f}")

# Compare results
print("\nPerformance comparison:")
print(f"Initial model accuracy: {evaluation_results['eval_accuracy']:.4f}")
print(f"PEFT model accuracy: {peft_evaluation_results['eval_accuracy']:.4f}")

PEFT model performance:
Accuracy: 0.9319

Performance comparison:
Initial model accuracy: 0.4998
PEFT model accuracy: 0.9319
